In [1]:
import pandas as pd
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
import re
import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

## Reading the Dataset

In [2]:
data = pd.read_csv("./spam.csv", encoding='latin-1').sample(frac=1).drop_duplicates()
data = data[['v1', 'v2']].rename(columns={"v1":"label", "v2":"text"})
data.head()

,label,text
1100,ham,You busy or can I come by at some point and fi...
5315,ham,Hahaha..use your brain dear
1236,ham,How much are we getting?
951,ham,Shb b ok lor... Thanx...
1085,ham,FR'NDSHIP is like a needle of a clock. Though ...


### Data Cleaning
#### Remove the Punctuations, stopwords, Lemmatize

In [3]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    return text

data['cleaned_text'] = data['text'].apply(lambda x: clean_text(x))
data.head()

,label,text,cleaned_text
1100,ham,You busy or can I come by at some point and fi...,busi come point figur tomorrow
5315,ham,Hahaha..use your brain dear,hahahaus brain dear
1236,ham,How much are we getting?,much get
951,ham,Shb b ok lor... Thanx...,shb b ok lor thanx
1085,ham,FR'NDSHIP is like a needle of a clock. Though ...,frndship like needl clock though v r clock v r...


In [4]:
data=data.drop(["text"],axis=1)
data.head()

,label,cleaned_text
1100,ham,busi come point figur tomorrow
5315,ham,hahahaus brain dear
1236,ham,much get
951,ham,shb b ok lor thanx
1085,ham,frndship like needl clock though v r clock v r...


## Test and Train Dataset Split

In [5]:
data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

## Use Glove Embeddings and Train the model

In [6]:
corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
#word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=100)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  """Entry point for launching an IPython kernel.


2020-02-05 12:11:34,496 Reading data from .
2020-02-05 12:11:34,497 Train: train.csv
2020-02-05 12:11:34,498 Dev: dev.csv
2020-02-05 12:11:34,499 Test: test.csv


C:\ProgramData\Anaconda3\lib\site-packages\flair\data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
C:\ProgramData\Anaconda3\lib\site-packages\flair\data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
C:\ProgramData\Anaconda3\lib\site-packages\flair\data_fetcher.py:463: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is mo

2020-02-05 12:11:40,781 Computing label dictionary. Progress:


100%|██████████████████████████████████████████████████████████████████████████| 4130/4130 [00:00<00:00, 344198.45it/s]


...

2020-02-05 12:18:14,507 epoch 36 - iter 91/130 - loss 0.02831085 - samples/sec: 516.15
2020-02-05 12:18:15,435 epoch 36 - iter 104/130 - loss 0.02845038 - samples/sec: 504.87
2020-02-05 12:18:16,405 epoch 36 - iter 117/130 - loss 0.03000542 - samples/sec: 479.28
2020-02-05 12:18:17,228 ----------------------------------------------------------------------------------------------------
2020-02-05 12:18:17,229 EPOCH 36 done: loss 0.0308 - lr 0.0004
2020-02-05 12:18:18,049 DEV : loss 0.04392469674348831 - score 0.9826
2020-02-05 12:18:18,073 BAD EPOCHS (no improvement): 2
2020-02-05 12:18:18,075 ----------------------------------------------------------------------------------------------------
2020-02-05 12:18:18,192 epoch 37 - iter 0/130 - loss 0.03444128 - samples/sec: 3617.59
2020-02-05 12:18:19,161 epoch 37 - iter 13/130 - loss 0.03070795 - samples/sec: 478.73
2020-02-05 12:18:20,080 epoch 37 - iter 26/130 - loss 0.02965951 - samples/sec: 507.95
2020-02-05 12:18:20,991 epoch 37 - ite

2020-02-05 12:19:19,650 EPOCH 42 done: loss 0.0300 - lr 0.0002
2020-02-05 12:19:20,468 DEV : loss 0.04379584640264511 - score 0.9826
Epoch    41: reducing learning rate of group 0 to 9.7656e-05.
2020-02-05 12:19:20,488 BAD EPOCHS (no improvement): 4
2020-02-05 12:19:20,490 ----------------------------------------------------------------------------------------------------
2020-02-05 12:19:20,491 ----------------------------------------------------------------------------------------------------
2020-02-05 12:19:20,492 learning rate too small - quitting training!
2020-02-05 12:19:20,493 ----------------------------------------------------------------------------------------------------
2020-02-05 12:19:24,579 ----------------------------------------------------------------------------------------------------
2020-02-05 12:19:24,582 Testing using best model ...
2020-02-05 12:19:24,583 loading file best-model.pt


C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:574: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


2020-02-05 12:19:27,941 0.9961	0.9961	0.9961
2020-02-05 12:19:27,943 
MICRO_AVG: acc 0.9923 - f1-score 0.9961
MACRO_AVG: acc 0.9809 - f1-score 0.9903
ham        tp: 458 - fp: 1 - fn: 1 - tn: 57 - precision: 0.9978 - recall: 0.9978 - accuracy: 0.9957 - f1-score: 0.9978
spam       tp: 57 - fp: 1 - fn: 1 - tn: 458 - precision: 0.9828 - recall: 0.9828 - accuracy: 0.9661 - f1-score: 0.9828
2020-02-05 12:19:27,944 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9961,
 'dev_score_history': [0.9865,
  0.9884,
  0.9826,
  0.9845,
  0.9865,
  0.9845,
  0.9865,
  0.9865,
  0.9865,
  0.9845,
  0.9845,
  0.9865,
  0.9865,
  0.9826,
  0.9807,
  0.9807,
  0.9845,
  0.9865,
  0.9826,
  0.9865,
  0.9826,
  0.9826,
  0.9826,
  0.9845,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826,
  0.9826],
 'train_loss_history': [0.15148688282531042,
  0.0848426594470556,
  0.07535099587761439,
  0.07040915398930128,
  0.06383458686849246,
  0.059092580570051303,
  0.049271565632751355,
  0.04867712014283125,
  0.04706459092692687,
  0.04353121120769244,
  0.04010398012514298,
  0.03722284267871426,
  0.036603732765294036,
  0.03572221935655062,
  0.033504460537089754,
  0.03420523538325842,
  0.032365413540257855,
  0.03432311093291411,
  0.03301021218156586,
  0.032221247313114314,
  0.03131773202465131,
  0.03045689035923435,
  